# Testing EDISON performance
The developer version of EDISON was installed using devtools:
```
devtools::install_github(repo='FrankD/EDISON',ref='MultipleTimeSeries', 
subdir='Package/EDISON/')
```

Frank Dondelinger provided the following script to test a small simulated dataset

In [ ]:
#### NOT RUN ####
library(EDISON)

set.seed(10)

# Generate a network with 5 nodes and no changepoints
test = generateNetwork(l=10, q=5, k_bar=0)

# Generate 100 repeated measurements for 10 time points
test.data = lapply(1:100, simulateNetwork, l=10, net=test)

# Make array and put dimensions in the right order (repeats, time points, variables)
test.data.array = sapply(test.data, function(x) x$sim_data, simplify='array')
test.data.array = aperm(test.data.array, c(3,2,1))

# Specify inference with no changepoints
edison.options = defaultOptions()
edison.options$cp.fixed = TRUE

# Run EDISON
edison.test = EDISON.run(test.data.array, num.iter=100000, options=edison.options)

# Calculate posterior probabilities of the edges in the network
calculateEdgeProbabilities(edison.test)$probs.segs

The code below was used to test a larger sample of simulated data, similar in size to the HILDA dataset, and measure the time it takes to run. 

In [1]:
# Some pointers to the HILDA data
setwd('~/Dropbox/HILDA/causal-models')
datadir <- ("~/Dropbox/HILDA/data/")

#### load some libraries ####
library(tidyverse) # needed for HILDA
library(haven)     # needed to load Stata files
library(EDISON)

#### and some helper functions for the HILDA data ####
source("~/Dropbox/HILDA/src/GetRaws.R")
source("~/Dropbox/HILDA/src/ZapLabel.R")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  2.0.1     ✔ dplyr   0.7.8
✔ tidyr   0.8.2     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: corpcor
Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

Loading required package: magrittr

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



#### Create simulated data with similar network size as HILDA example (see below)

In [2]:
set.seed(10)

# Generate a network with 9 nodes and no changepoints
test = generateNetwork(l=16, q=9, k_bar=0)

# Generate 1000 repeated measurements for 16 time points
test.data = lapply(1:1000, simulateNetwork, l=16, net=test)

# Make array and put dimensions in the right order (repeats, time points, variables)
test.data.array = sapply(test.data, function(x) x$sim_data, simplify='array')
test.data.array = aperm(test.data.array, c(3,2,1))

# Specify inference with no changepoints
edison.options = defaultOptions()
edison.options$cp.fixed = TRUE

In [3]:
# Run and time EDISON
start.time <- Sys.time()
edison.test = EDISON.run(test.data.array, num.iter=10000, options=edison.options)
end.time <- Sys.time()
print(end.time - start.time)

[1] "Initialisation successful."
[1] "Starting tvDBN iterations..."
5 % 10 % 15 % 20 % 25 % 30 % 35 % 40 % 45 % 50 % 55 % 60 % 65 % 70 % 75 % 80 % 85 % 90 % 95 % 100 % [1] ""
[1] "End of iterations"
Time difference of 1.228027 days


In [4]:
# Calculate posterior probabilities of the edges in the network
calculateEdgeProbabilities(edison.test)$probs.segs

0,0.000,0,0.000,0.000,0.071,0.122,0.000,0.000
0,1.000,0,0.000,0.000,0.113,0.000,0.000,0.000
0,0.000,0,0.000,0.000,0.090,0.887,0.000,0.000
0,0.031,0,1.000,0.000,0.045,0.000,0.000,0.000
0,0.000,0,1.000,0.000,0.105,0.094,0.032,0.000
0,0.021,0,0.006,0.004,0.026,0.000,1.000,1.000
0,0.049,0,0.000,0.000,0.052,0.125,0.000,0.051
1,0.000,1,0.000,0.000,0.142,1.000,0.000,0.000
0,0.000,0,0.018,1.000,0.031,0.000,0.000,0.000


It took **30 hrs** to test a network with p = 9, n = 1000 and no changepoints with simulated data. 
To be done: Compare the edge probabilities with the edges in the simulated data to check accuracy.

## Testing the HILDA data  
The aim here is only to check whether real data from HILDA with the same dimensions takes a similar time to run as the simulated data (i.e,. 30 hrs or 1.22 days). This is provided as just a check to test whether there is something funny about the HILDA data causing the problem. 

In [5]:
#### Load the HILDA dataset ####
filepaths <- list.files(
  path = datadir,
  pattern = '^Combined.*.dta$',
  full.names = TRUE
)

cat('Importing dataframes')
hilda <- list()
for (pathtofile in filepaths) {
  df <- read_dta(pathtofile)
  hilda <- append(hilda, list(df))
  cat('.')
}
cat('Done')

Importing dataframes................Done

In [ ]:
#### Create the variable matrices ####
predictors <- c('ghmh',       # mental health
                'losat',      # life satisfaction
                'losatyh',    # health satisfaction
                'wscei',      # weekly gross wages/salary
                'lssupac',    # I don't have anyone I can confide in
                'lssupvl',    # I often feel very lonely
                'lsrush',     # feeling rushed
                'lshrhw',     # hours per week housework
                'lsshare'     # share of housework
)

df.long <- GetRaws(hilda, predictors)

matrixlist <- list()
for (code in predictors) {
  df.long %>%
    filter(hildacode == code) %>%
    spread(waveid, value) %>%
    select(-hildacode, -xwaveid) %>%
    as.matrix() -> mat
  matrixlist[[code]] <- mat
}

hilda.array <- simplify2array(matrixlist)
write_rds(hilda.array, 'data/hildadata.rds')

We have 2917 people in the selected data set. Test a random selection of 1000 people

In [2]:
hilda.array <- read_rds('data/hildadata.rds')

# Specify inference with no changepoints
edison.options = defaultOptions()
edison.options$cp.fixed = TRUE

# Run EDISON
randomidx <- sample.int(n = 2917, size = 1000) # random selection of row
hilda.random <- hilda.array[randomidx, , ]

start.time <- Sys.time()
hilda.test = EDISON.run(hilda.random, num.iter=10000, options=edison.options)
end.time <- Sys.time()
print(end.time - start.time) # 1.461462 mins

[1] "Initialisation successful."
[1] "Starting tvDBN iterations..."
5 % 10 % 15 % 20 % 25 % 30 % 35 % 40 % 45 % 50 % 55 % 60 % 65 % 70 % 75 % 80 % 85 % 90 % 95 % 100 % [1] ""
[1] "End of iterations"
Time difference of 1.461462 mins


Original time difference of 1.248522 days.  
Current time difference of 1.461462 mins.

In [3]:
calculateEdgeProbabilities(hilda.test)$probs.segs

# [[1]]
#      [,1] [,2] [,3] [,4] [,5] [,6] [,7]  [,8]  [,9]
# [1,]    1    0    1    0    1    1    1 0.003 0.037
# [2,]    1    1    0    0    0    1    0 0.251 0.000
# [3,]    0    0    0    1    1    0    0 0.000 0.308
# [4,]    0    0    0    1    0    1    1 1.000 0.115
# [5,]    1    1    1    0    1    1    0 0.000 0.174
# [6,]    1    1    1    0    1    0    0 0.023 0.767
# [7,]    0    1    1    1    0    0    1 0.635 0.341
# [8,]    0    0    0    1    0    0    1 1.000 1.000
# [9,]    0    0    0    0    0    0    0 1.000 1.000

#write_rds(hilda.test, 'data/hildaresult1.rds')

0,1,0,1,1,1,1,0,0
1,1,0,0,1,1,0,0,1
1,0,0,1,0,0,0,0,0
0,0,1,0,0,0,1,1,0
1,0,1,0,1,1,0,0,1
0,1,0,0,1,1,0,0,1
0,1,1,1,0,0,1,1,0
1,0,0,0,0,0,0,1,1
0,0,1,1,0,0,1,1,0


#### Test a different set of 1000 people from HILDA

In [4]:
# Take the next sample
rowidx <- seq(1:2917)
newrandomidx <- sort(sample(rowidx[!(rowidx %in% randomidx)], 1000))
sum(randomidx %in% newrandomidx) # check samples are unique

[1] 0

In [5]:
hilda.random <- hilda.array[newrandomidx, , ]

start.time <- Sys.time()
hilda.test = EDISON.run(hilda.random, num.iter=10000, options=edison.options)
end.time <- Sys.time()
print(end.time - start.time)

[1] "Initialisation successful."
[1] "Starting tvDBN iterations..."
5 % 10 % 15 % 20 % 25 % 30 % 35 % 40 % 45 % 50 % 55 % 60 % 65 % 70 % 75 % 80 % 85 % 90 % 95 % 100 % [1] ""
[1] "End of iterations"
Time difference of 1.47213 mins


In [6]:
calculateEdgeProbabilities(hilda.test)$probs.segs

# 0.946 1 1 0 1 1 1 0 0.000
# 0.000 0 0 0 0 0 0 1 0.045
# 0.000 0 0 1 1 1 0 0 1.000
# 0.000 0 1 1 0 0 1 1 0.000
# 1.000 1 1 0 1 1 0 0 0.000
# 1.000 0 0 0 1 0 0 0 0.691
# 1.000 1 0 1 0 0 1 1 0.260
# 0.000 1 0 1 0 0 1 0 1.000
# 0.045 0 1 0 0 1 0 1 1.000


1,0,0.891,0,1,0,0,1,0
0,1,1.000,1,1,1,1,1,0
1,0,1.000,1,0,0,0,0,1
0,0,0.000,0,0,1,1,1,1
1,1,0.707,0,1,1,0,0,0
1,1,0.387,1,1,0,0,0,1
0,1,0.000,1,0,0,1,0,1
0,0,0.002,0,0,1,0,0,0
0,0,0.000,0,0,0,1,1,0


In [7]:
write_rds(hilda.test, 'data/hildaresult2.rds')

## Conclusions
With n = 1000 and p = 9, EDISON takes about 30 hours on my machine. I also ran it on the HILDA dataset with the same dimensions and it also takes about 30 hours (so nothing funny going on between simulated and real data).  

However after the fix provided by SIH, the code takes about 1.4 minutes to run on my machine. This represents a performance improvement over 1000 times faster than the original code.